In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
from PyNomaly import loop
import string

In [2]:
columns = {"letter": "capital letter	(26 values from A to Z)", "x-box": "horizontal position of box(integer)",
           "y-box": "vertical position of box(integer)", "width": "width	width of box(integer)",
           "high": "height of box(integer)", "onpix": "onpix	total", "x-bar": "mean x of on pixels in box(integer)",
           "y-bar": "mean y of on pixels in box(integer)", "x2bar": "mean x variance(integer)", "y2bar": "mean y variance	(integer)",
           "xybar": "mean x y correlation	(integer)", "x2ybr": "mean of x * x * y	(integer)", "xy2br": "mean of x * y * y (integer)",
           "x-ege": "mean edge count left to right (integer)", "xegvy": "correlation of x-ege with y	(integer)",
           "y-ege": "ean edge count bottom to top	(integer)", "yegvx": "correlation of y-ege with x	(integer)"}


path = Path("C:/Users/Dani/Documents/temp/kddm2\Data/letter-recognition.data")

alphabet = list(string.ascii_uppercase)

In [3]:
df_letters = pd.read_csv(path, names=list(columns.keys()))

dataframes = dict()

In [4]:
sum = 0


for letter in alphabet:
    df = df_letters.loc[(df_letters.letter == letter), :]
    df = df.drop('letter', axis=1, inplace=False)
    sum += len(df.index)
    dataframes[letter] = df

print(sum)

20000


In [5]:
amount =  len(df_letters.index)
amount

20000

In [6]:
means = df_letters.groupby("letter").mean()
print(df_letters)

      letter  x-box  y-box  width  high  onpix  x-bar  y-bar  x2bar  y2bar  \
0          T      2      8      3     5      1      8     13      0      6   
1          I      5     12      3     7      2     10      5      5      4   
2          D      4     11      6     8      6     10      6      2      6   
3          N      7     11      6     6      3      5      9      4      6   
4          G      2      1      3     1      1      8      6      6      6   
...      ...    ...    ...    ...   ...    ...    ...    ...    ...    ...   
19995      D      2      2      3     3      2      7      7      7      6   
19996      C      7     10      8     8      4      4      8      6      9   
19997      T      6      9      6     7      5      6     11      3      7   
19998      S      2      3      4     2      1      8      7      2      6   
19999      A      4      9      6     6      2      9      5      3      1   

       xybar  x2ybr  xy2br  x-ege  xegvy  y-ege  yegvx  
0     

In [20]:
extents = [1,2,3] # number of standarddivations, the value has to diviate
n_neighbors =  range(1,21)
critical_values = np.arange(0.80, 1, 0.01)

In [28]:
rows = alphabet.append("sum")
optimization_results = pd.DataFrame()
optimization_results["letter"] = alphabet

In [33]:
for extent in extents:
    for n in n_neighbors:
        for value in critical_values:
            column = f"e{extent}_n{n}_{value*100}"
            optimization_results[column] = 0
            outliers = 0

            part_outliers = dict()

            for letter, dataframe in dataframes.items():
                m = loop.2(dataframe, extent=extent, n_neighbors=n).fit()
                scores = list(m.local_outlier_probabilities)
                part_outlier_number = 0
                for score in scores:
                    if score >= value:
                        part_outlier_number += 1
                outliers += part_outlier_number
                percentage_outlier = part_outlier_number/len(dataframe.index)
                optimization_results.loc[optimization_results.letter == letter, column] = percentage_outlier
                
                part_outliers[letter] = {"part outliers":part_outlier_number, "Percentage outlier": percentage_outlier, "max value": max(scores)}

            print(f"Results for extent = {extent}, n_neighbors= {n} and critical value = {value}")
            optimization_results.loc[optimization_results.letter == "sum", column] = outliers/amount
            print(optimization_results.loc[:, ["letter", column]])

Results for extent = 1, n_neighbors= 1 and critical value = 0.8
   letter  e1_n1_80.0
0       A    0.038023
1       B    0.049608
2       C    0.035326
3       D    0.054658
4       E    0.059896
5       F    0.020645
6       G    0.046572
7       H    0.029973
8       I    0.103311
9       J    0.025435
10      K    0.028417
11      L    0.068331
12      M    0.059343
13      N    0.052363
14      O    0.038513
15      P    0.019925
16      Q    0.024266
17      R    0.026385
18      S    0.029412
19      T    0.061558
20      U    0.049200
21      V    0.064136
22      W    0.034574
23      X    0.049555
24      Y    0.044529
25      Z    0.073569
26    sum    0.045700
Results for extent = 1, n_neighbors= 1 and critical value = 0.81
   letter  e1_n1_81.0
0       A    0.038023
1       B    0.049608
2       C    0.035326
3       D    0.054658
4       E    0.059896
5       F    0.020645
6       G    0.046572
7       H    0.029973
8       I    0.103311
9       J    0.025435
10      K    

In [35]:
optimization_results.to_csv("local_outlier_probability_hyper_parameter_testing.csv")